# Regressão Linear
> Encontrando a melhor linha

- toc: true 
- badges: true
- comments: true
- categories: [machine learning, aprendizado supervisionado]
<!-- - image: images/chart-preview.png -->

## TL;DR

Regressão linear é um modelo supervisionado de machine learning, que busca encontrar a linha que melhor representa um grupo de pontos.

---

# Senta que lá vem história...

Quando eu morava nos Estados Unidos, estudando engenharia mecânica, eu tive um curso sobre experiências - como as incertezas das medidas vinda dos aparelhos se apresentavam nos resultados. Não muito importante. Mas no experimento, eu e meu parceiro (vamos chamá-lo de Túlio) deveríamos pesar um [béquer](https://pt.wikipedia.org/wiki/B%C3%A9quer) com um líquido várias vezes (com diferentes volumes) e decidir se esse líquido era de água ou não - através do post, vamos supor sim.

Eu sabia que independentemente do que fosse, a gente deveria ver uma linha de pontos, já que todos líquidos têm uma densidade constante, e a peso total é proporcional ao volume $peso_{total} = densidade \cdot volume + peso_{béquer}$. Mas para a minha surpresa não foi bem isso que observamos.

<!-- <div style="text-align:center"><img src="" /></div> --> - manim: expectativa vs realidade

**Porquê?**

A balança não é exata, e quando medimos o volume também cometemos erros. Quando levamos tudo me consideração, isso faz diferença dependendo da sensibilidade do nosso experimento. Essas erros aleatórios que variam, normalmente são pequenos e não podemos prever chamamos de [ruído](https://pt.wikipedia.org/wiki/Ru%C3%ADdo). Mas mesmo com esse ruído todo, é possível encontrar os pontos se não tivéssemos ruído nenhum?

## O setup

Estamos tentando aproximar a peso de um béquer com água, sendo que sabemos a peso do béquer {% fn 2 %}. Ou seja:

* $P \leftarrow$ funcao do peso total do liquido no becker
* $\rho \leftarrow$ densidade do liquido
* $P_{becker} \leftarrow$ peso do becker

$$P(V) = \rho V + P_{becker}$$

> Tip: Fique atento às letras aqui e o que elas significam, porque vamos usá-las ao longo do post

Isso é o que queremos aproximar, mas também observamos o ruído. Com o ruído, a equação pode ser escrita assim:

* $\epsilon \leftarrow$ ruído (um pequeno erro, aleatório)

$$P(V) = \rho V + P_{becker} + \epsilon$$

## Abordagem ingênua

A primeira coisa que eu pensei foi: vou no olho! Quão difícil pode ser?

<!-- <div style="text-align:center"><img src="" /></div> --> - deal with muri

Maaaaas, o Tulio sempre complicava tudo... A linha que ele traçou era um pouco diferente, e ele falava que era a certa. Agora, veja bem: eu sabia que a minha linha era uma aproximação melhor, mas se voce conhece o tuliao, vai saber que ele é bem teimoso. Como que eu posso provar pra ele que a minha linha era melhor? Como que a gente compara duas linhas?

## A funcao do erro

Uma ideia que surgiu era: e se a gente pegar a distancia vertical de cada ponto? Ou seja - se a gente pegar cada volume (x) que a gente mediu na balanca e comparar com o peso que a nossa linha mostra? E somar todos? Quem tiver a menor soma, tem a melhor linha (e o orgulho intacto)!

Eu logo sugeri pegar a diferenca ao quadrado, porque assim a gente nao precisa se preocupar com "diferenças negativas" e "diferenças positivas" cancelando uma a outra.

Depois de calcular os numeros, a gente tambem decidiu dividir pelo numero de pontos que temos - isso nao faz diferenca para nos dois, mas assim a gente poderia comparar a nossa aproximacao com a dos outros grupos, ja que eles tinham um numero diferente de medidas.

<!-- <div style="text-align:center"><img src="" /></div> --> - MSE

Depois de toda a discussao, a gente percebeu que isso tudo que a gente fez nada mais é do que [erro quadrático médio](https://pt.wikipedia.org/wiki/Erro_quadr%C3%A1tico_m%C3%A9dio), e que também que essa medida nos da uma interpretação de quao boa a nossa linha é.

## Como melhorar o nosso chute?

Depois disso tudo, fiquei pensando: "nosso erro depende da linha e a gente definiu uma funcao matematica pra isso, nao tem como a gente melhorar mais ainda a aproximacao? Chegar na melhor linha possivel?". Ai eu lembrei que se a gente tem essa funcao do erro a gente pode iterativamente melhorar a aproximacao, com algoritmos como o [gradient descent](todo_grad), desde que ela seja [difernciavel](todo_diff).

<!-- <div style="text-align:center"><img src="" /></div> --> - grad desc

Vamos ver como é que fica.

## A nossa linha

Antes de falar como fazemos pra melhorar a nossa aproximação, vamos definir a nossa linha:

* $P \leftarrow$ funcao do peso total do liquido no becker
* $\rho \leftarrow$ densidade do liquido
* $P_{becker} \leftarrow$ peso do becker

$$P(V) = \rho V + P_{becker}$$

E, de maneira mais formal, o que eu e o Tulio estavamos encontrando são os melhores valores de $\rho$ e $P_{becker}$, já que são esses valores que definem a linha. E a funcao do erro fica:

* $E \leftarrow$ funcao do erro da nossa linha baseado nas nossas observacoes
* $V_{obs} \leftarrow$ peso total observado
* $P_{obs} \leftarrow$ peso total observado
* $N \leftarrow$ numero de observacoes


$$E(P_{becker},\rho) =  \frac{1}{N}\sum_{obs=1}^{N}{(P(V_{obs}) - P_{obs})^2}$$

Eu sei que é mais intimidador quando colocamos tudo de uma vez em uma equacao. Mas lembra que essa equaçao esta descrevendo nada mais do que discutimos [acima](##A-funcao-do-erro). Tome seu tempo pra verificar que voce entende o que ta acontecendo aqui.

<!-- Por exemplo, o símbolo $\sum_{obs=1}^{N}$ nada mais diz que vamos somar desde a primeira observação ($obs=1$) até a observação $N$ (que é a ultima observação) - ou seja, soma a diferença ao quadrado pra todas as observacoes ($(P(V_{obs}) - P_{obs})^2$).
 -->
Agora, como que a gente pode ajustar os valores de $\rho$ e $P_{becker}$ pra melhorar nossa aproximação?

## Aplicando o gradient descent

Tudo o que precisamos fazer é definir uma função (que é diferenciável) que descreve o **erro** da nossa aproximação. Com essa função de erro nos podemos aplicar algoritmos de otimização para reduzir esse erro. Uma maneira simples é usando o [gradient descent](todo_grad) (descida do gradiente).

Em gradient descent, a gente reduz o erro dando um passo na direção oposta do gradiente - ou seja, na direção oposta da derivada em cada dimensão{% fn 1 %}. Como que fica o update de cada parâmetro?

Mas antes de mergulharmos nas letrinhas, queria lembrar que essa provavelmente vai ser a parte mais confusa, especialmente se essa é a primeira vez que você está vendo isso. Mas vamos com calma. Vamos definir alguns termos daqui a pouco, mas também vamos explicá-los um por um, e até resolver um exemplo. Até o fim desse post vai ficar mais claro. E fique à vontade para ler, pensar e reler, até que você fique confortável.

> Warning: Matemática à frente

* GIF com as transformacoes do formato do video ate o update final

Ou entao, num passo a passo mais detalhado (que visualmente parece mais complicado do que é):

As derivadas parciais da nossa função de erro:

> Note: Lembre-se que a derivada parcial $\frac{\partial f}{\partial x}(x,y)$ é coeficiente da linha tangente a $f$ se $y$ fosse um número constante.

$$\frac{\partial E}{\partial P_{becker}}(P_{becker},\rho) =  \frac{2}{N}\sum_{obs=1}^{N}{(P(V_{obs}) - P_{obs})} \cdot \frac{\partial P}{\partial P_{becker}}(V_{obs})$$

$$\frac{\partial E}{\partial \rho}(P_{becker},\rho) =  \frac{2}{N}\sum_{obs=1}^{N}{(P(V_{obs}) - P_{obs})} \cdot \frac{\partial P}{\partial \rho}(V_{obs})$$

O passo para cada dimensão:

* $\alpha \leftarrow$ a taxa de aprendizado

$$P_{becker} \leftarrow P_{becker} - \alpha \cdot \frac{\partial E}{\partial P_{becker}}(P_{becker},\rho)$$

$$\rho \leftarrow P_{becker} - \alpha \cdot \frac{\partial E}{\partial\rho}(P_{becker},\rho)$$

E colocando tudo junto:

$$P_{becker} \leftarrow P_{becker} - \alpha \cdot \frac{2}{N}\sum_{obs=1}^{N}{(P(V_{obs}) - P_{obs})} \cdot \frac{\partial P}{\partial P_{becker}}(V_{obs})$$

$$\rho \leftarrow P_{becker} - \alpha \cdot \frac{2}{N}\sum_{obs=1}^{N}{(P(V_{obs}) - P_{obs})} \cdot \frac{\partial P}{\partial \rho}(V_{obs})$$


<div style="text-align:center"><img src="https://media.giphy.com/media/Ni4cpi0uUkd6U/giphy.gif" width="20%"/></div>

> Tip: Eu sei que é muita letra, e muito confuso. Mas lembre-se que a única coisa que estamos fazendo aqui é reduzir o valor de uma função (função do erro). O porquê o algoritmo funciona, ou a intuição por trás do algoritmo não está no escopo desse post. Mas fique à vontade para ler o post sobre [gradient descent](todo_grad) onde mergulhamos mais a fundo.

put a `#collapse-show` flag at the top of any cell if you want to **show** that cell by default, but give the reader the option to hide it:

In [4]:
#collapse-show
cars = 'https://vega.github.io/vega-datasets/data/cars.json'
movies = 'https://vega.github.io/vega-datasets/data/movies.json'
sp500 = 'https://vega.github.io/vega-datasets/data/sp500.csv'
stocks = 'https://vega.github.io/vega-datasets/data/stocks.csv'
flights = 'https://vega.github.io/vega-datasets/data/flights-5k.json'

## Interactive Charts With Altair

Charts made with Altair remain interactive.  Example charts taken from [this repo](https://github.com/uwdata/visualization-curriculum), specifically [this notebook](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb).

In [5]:
# hide
df = pd.read_json(movies) # load movies data
genres = df['Major_Genre'].unique() # get unique field values
genres = list(filter(lambda d: d is not None, genres)) # filter out None values
genres.sort() # sort alphabetically

In [4]:
#hide
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

### Example 1: DropDown

In [5]:
# single-value selection over [Major_Genre, MPAA_Rating] pairs
# use specific hard-wired values as the initial selected values
selection = alt.selection_single(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    init={'Major_Genre': 'Drama', 'MPAA_Rating': 'R'},
    bind={'Major_Genre': alt.binding_select(options=genres), 'MPAA_Rating': alt.binding_radio(options=mpaa)}
)
  
# scatter plot, modify opacity based on selection
alt.Chart(movies).mark_circle().add_selection(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)

alt.Chart(...)

### Example 2: Tooltips

In [6]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales', encodings=['x'])
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use min extent to stabilize axis title placement
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

### Example 3: More Tooltips

In [7]:
# select a point for which to provide details-on-demand
label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

# define our base line chart of stock prices
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # base line chart
    
    # add a rule mark to serve as a guide line
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),
    
    # add circle marks for selected time points, hide unselected points
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # add white stroked text to provide a legible background for labels
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # add text labels for stock prices
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),
    
    data=stocks
).properties(
    width=700,
    height=400
)

alt.LayerChart(...)

## Data Tables

You can display tables per the usual way in your blog:

In [11]:
movies = 'https://vega.github.io/vega-datasets/data/movies.json'
df = pd.read_json(movies)
# display table with pandas
df[['Title', 'Worldwide_Gross', 
    'Production_Budget', 'Distributor', 'MPAA_Rating', 'IMDB_Rating', 'Rotten_Tomatoes_Rating']].head()

,Title,Worldwide_Gross,Production_Budget,Distributor,MPAA_Rating,IMDB_Rating,Rotten_Tomatoes_Rating
0,The Land Girls,146083.0,8000000.0,Gramercy,R,6.1,NaN
1,"First Love, Last Rites",10876.0,300000.0,Strand,R,6.9,NaN
2,I Married a Strange Person,203134.0,250000.0,Lionsgate,None,6.8,NaN
3,Let's Talk About Sex,373615.0,300000.0,Fine Line,None,NaN,13.0
4,Slam,1087521.0,1000000.0,Trimark,R,3.4,62.0


## Images

### Local Images

You can reference local images and they will be copied and rendered on your blog automatically.  You can include these with the following markdown syntax:

`![](my_icons/fastai_logo.png)`

![](my_icons/fastai_logo.png)

### Remote Images

Remote images can be included with the following markdown syntax:

`![](https://image.flaticon.com/icons/svg/36/36686.svg)`

![](https://image.flaticon.com/icons/svg/36/36686.svg)

### Animated Gifs

Animated Gifs work, too!

`![](https://upload.wikimedia.org/wikipedia/commons/7/71/ChessPawnSpecialMoves.gif)`

![](https://upload.wikimedia.org/wikipedia/commons/7/71/ChessPawnSpecialMoves.gif)

### Captions

You can include captions with markdown images like this:

```
![](https://www.fast.ai/images/fastai_paper/show_batch.png "Credit: https://www.fast.ai/2020/02/13/fastai-A-Layered-API-for-Deep-Learning/")
```


![](https://www.fast.ai/images/fastai_paper/show_batch.png "Credit: https://www.fast.ai/2020/02/13/fastai-A-Layered-API-for-Deep-Learning/")





# Other Elements

## Tweetcards

Typing `> twitter: https://twitter.com/jakevdp/status/1204765621767901185?s=20` will render this:

> twitter: https://twitter.com/jakevdp/status/1204765621767901185?s=20

## Youtube Videos

Typing `> youtube: https://youtu.be/XfoYk_Z5AkI` will render this:


> youtube: https://youtu.be/XfoYk_Z5AkI

## Boxes / Callouts 

Typing `> Warning: There will be no second warning!` will render this:


> Warning: There will be no second warning!



Typing `> Important: Pay attention! It's important.` will render this:

> Important: Pay attention! It's important.



Typing `> Tip: This is my tip.` will render this:

> Tip: This is my tip.



Typing `> Note: Take note of this.` will render this:

> Note: Take note of this.



Typing `> Note: A doc link to [an example website: fast.ai](https://www.fast.ai/) should also work fine.` will render in the docs:

> Note: A doc link to [an example website: fast.ai](https://www.fast.ai/) should also work fine.

## Footnotes

You can have footnotes in notebooks, however the syntax is different compared to markdown documents. [This guide provides more detail about this syntax](https://github.com/fastai/fastpages/blob/master/_fastpages_docs/NOTEBOOK_FOOTNOTES.md), which looks like this:

```
{% raw %}For example, here is a footnote {% fn 1 %}.
And another {% fn 2 %}
{{ 'This is the footnote.' | fndetail: 1 }}
{{ 'This is the other footnote. You can even have a [link](www.github.com)!' | fndetail: 2 }}{% endraw %}
```

For example, here is a footnote {% fn 1 %}.

And another {% fn 2 %}

{{ 'This is the footnote.' | fndetail: 1 }}
{{ 'This is the other footnote. You can even have a [link](www.github.com)!' | fndetail: 2 }}

## Footnotes

{{ 'Veja o post sobre [gradient descent](todo_grad_desc) para mais detalhes.' | fndetail: 1 }}
{{ 'Mais precisamente, deveríamos estar falando de *massa* ao invés de *peso*. Mas no nosse approach isso não faz diferença.' | fndetail: 1 }}